## Preprocessing Phase

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import sys

from wfdb.processing import resample_multichan, find_local_peaks, compare_annotations

from scipy.signal import convolve

from wfdb import wrann, rdann

from io import StringIO

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense, TimeDistributed, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.metrics import mean_absolute_error

import warnings

warnings.filterwarnings('ignore')

In [ ]:
os.chdir('Unpreprocessed_Data/')

In [ ]:
def parabola(a,n,r = 4): 
    '''    
    n : dimension of the segment to held the parables
    a : list object of the positions of the QRS complexes
    r : half amplitude of the parables, that is 4 by default
    
    '''
    assert n>2*r
    
    y = np.zeros(n, dtype = np.float32)
    
    x= np.array(range(2,2*r+1))
    
    for i in a:
        
        if i > r-1 and i <= n-r:
            
            y[i-r+1:i+r] = ((r+1)**2-(x-r-1)**2)/(r+1)**2
            
        elif i < r:
            
            y[:i+r] = ((r+1)**2-(x[r-i-1:]-r-1)**2)/(r+1)**2
        
        elif i<n:
            
            y[i-r+1:] = ((r+1)**2-(x[:r-1+(n-i)]-r-1)**2)/(r+1)**2
    
    return y

In [ ]:
Identifier_list = list()

for element in range(len(os.listdir())):
    
    Identifier_list.append(os.listdir()[element][0:3])
    
Identifier_list = list(dict.fromkeys(Identifier_list))


for _id in Identifier_list:
    
    if _id.startswith('I'):
        
        channels = ['II', 'V1']
        
        file_type = 'Training File'
    
    else:
        
        channels = ['MLII', 'V5']
        
        file_type = 'Testing File'
        
    
    Annot = wfdb.rdann(_id, extension = 'atr')
    
    Sig, Head = wfdb.rdsamp(_id, channel_names = channels)
        
    Sig, Annot = resample_multichan(Sig, Annot, Head['fs'], 100) 
    
    Weights = np.repeat(1.0, 100) / 100 
        
    Conv_Sig_0 = convolve(Sig[:,0], Weights, mode = 'same') 
    Conv_Sig_1 = convolve(Sig[:,1], Weights, mode = 'same') 
        
    Final_Sig_0 = Sig[:,0] - Conv_Sig_II 
    Final_Sig_1 = Sig[:,1] - Conv_Sig_V1
        
    Final_Sig = np.vstack([Final_Sig_0, Final_Sig_1])
    Final_Sig = np.transpose(Final_Sig)
    
    signal_list = ['N', 'L', 'R', 'B', 'A', 'a', 'J', 'S', 'V', 'r', 'F', 'e', 'j', 'n', 'E', '/', 'f', 'Q', '?']
    QRS_Index = [pos for pos, signal in enumerate(Annot.symbol) if signal in signal_list] 
    QRS_Pos = [int(list(Annot.sample)[i]) for i in QRS_Index] 
        
    Target_Seq = parabola(QRS_Pos, Sig.shape[0]) 
        
    Sig_Target = np.insert(Final_Sig, 2, Target_Seq, axis = 1) 
        
    np.save(str(_id), Sig_Target)
    
    print(file_type + _id + ' done')


## Training Phase

In [ ]:
os.chdir('Preprocessed_Data/')

In [ ]:
Training_records, Testing_records = 75, 48

Training_array = np.zeros((int(Training_records * 0.75) + 2, 3, 180000), dtype = np.float32)

Validation_array = np.zeros((int(Training_records * 0.25) + 1, 3, 180000), dtype = np.float32)

Testing_array = np.zeros((Testing_records, 3, 180555), dtype = np.float32 )

In [ ]:
train_count, test_count = 0, 0

for _id in range(len(os.listdir())):
    
    file = os.listdir()[_id]
    
    if file.startswith('I'):
        
        record = np.transpose(np.load(file))
        
        if train_count <= Validation_array.shape[0] - 1:
        
            Validation_array[train_count,:,:] = record
            train_count += 1
            
        else:
            
            Training_array[train_count - 18,:,:] = record
            train_count += 1
    else:
        
        record = np.transpose(np.load(file))
        Testing_array[test_count,:,:] = record
        
        test_count += 1       

In [ ]:
def Select_ECG_portion(array_3D, seqL, ninputs):
    
    record_id = tf.random.uniform([1], minval = 0, maxval = array_3D.shape[0], dtype = tf.dtypes.int32)
    
    array_2D = array_3D[record_id[0],:,:]
        
    record_seg_dim = seqL * ninputs
    
    record_seg = tf.random_crop(array_2D, [3, record_seg_dim])
    
    inputs_ch1 = record_seg[0,:]
    inputs_ch2 = record_seg[1,:]
    
    inputs = tf.concat([inputs_ch1, inputs_ch2], axis = 0)
    target = record_seg[2,:]
    
    return inputs, target

## Feedforward Net

In [ ]:
seqL, fs, time_step = 20, 100, 0.5

ninputs = int(time_step * fs)

batchSize = 8

Train = tf.data.Dataset.from_tensor_slices(Training_array)
Train = Train.map(lambda x:  Select_ECG_portion(x,seqL, ninputs))
Train = Train.repeat()
Train = Train.batch(batchSize)

Validation = tf.data.Dataset.from_tensor_slices(Validation_array)
Validation = Validation.map(lambda x:  Select_ECG_portion(x,seqL, ninputs))
Validation = Validation.repeat()  
Validation = Validation.batch(batchSize)

Test = tf.data.Dataset.from_tensor_slices(Testing_array)
Test = Test.map(lambda x:  Select_ECG_portion(x,seqL, ninputs))
Test = Test.repeat()  
Test = Test.batch(batchSize)

In [ ]:
Dense_units = 2000

ff_Model = Sequential()

ff_Model.add(Dense(Dense_units, activation = 'relu',input_shape = (2*seqL*ninputs,)))

ff_Model.add(Dense(Dense_units, activation = 'relu'))
ff_Model.add(Dense(Dense_units, activation = 'relu'))
ff_Model.add(Dense(Dense_units, activation = 'relu'))
ff_Model.add(Dense(Dense_units, activation = 'relu'))

ff_Model.add(Dense(seqL*ninputs))

In [ ]:
ff_Model.compile(optimizer = Adam(0.001), loss = mean_squared_error, metrics = [ mean_absolute_error ])
ff_Model.fit(Train, epochs = 90, steps_per_epoch = 1500, validation_data = Validation, validation_steps = 100, verbose = 2)

In [ ]:
out = ff_Model.evaluate(Test, steps = 1000)

print('test mean square error (loss): ', out[0], '  test absolute error: ', out[1])

iterator = Test.make_initializable_iterator()

next_element = iterator.get_next()

with tf.Session() as sess:
    
    sess.run(iterator.initializer)
    
    inp, targ = sess.run(next_element)
    
output = ff_Model.predict(inp)

t = range(1000)

plt.plot(t,inp[5,:1000],'k',t,targ[5,:1000]-2,'r',t,output[5,:1000]-2,'b')

plt.show()

## Recurrent Net

In [ ]:
seqL, fs, time_step = 35, 100, 0.3

ninputs = int(time_step * fs)

batchSize = 8

Train = tf.data.Dataset.from_tensors(Training_array)
Train = Train.map(lambda x:  Select_ECG_portion(x,seqL, ninputs))
Train = Train.repeat()
Train = Train.batch(batchSize)

Validation = tf.data.Dataset.from_tensors(Validation_array)
Validation = Validation.map(lambda x:  Select_ECG_portion(x,seqL, ninputs))
Validation = Validation.repeat()  
Validation = Validation.batch(batchSize)

Test = tf.data.Dataset.from_tensors(Testing_array)
Test = Test.map(lambda x:  Select_ECG_portion(x,seqL, ninputs))
Test = Test.repeat()  
Test = Test.batch(batchSize)

In [ ]:
LSTM_units = 212

denseDim = ninputs


rr_Model = tf.keras.Sequential()

rr_Model.add(Reshape((seqL,2*ninputs), input_shape = (2*seqL*ninputs,)))

rr_Model.add(LSTM(units = LSTM_units, return_sequences = True))
rr_Model.add(LSTM(units = LSTM_units, return_sequences = True))
rr_Model.add(LSTM(units = LSTM_units, return_sequences = True))
rr_Model.add(LSTM(units = LSTM_units, return_sequences = True))
rr_Model.add(LSTM(units = LSTM_units, return_sequences = True))

rr_Model.add(TimeDistributed(Dense(denseDim)))

rr_Model.add(Reshape((seqL*ninputs,)))

In [ ]:
rr_Model.compile(optimizer = Adam(0.01), loss = mean_squared_error,metrics = [mean_absolute_error])
rr_Model.fit(Train, epochs = 130, steps_per_epoch = 1500, validation_data = Validation, validation_steps = 100, verbose = 2)

In [ ]:
out = rr_Model.evaluate(Test, steps = 1000)

print('test mean square error (loss): ', out[0], '  test absolute error: ', out[1])

iterator = Test.make_initializable_iterator()

next_element = iterator.get_next()

with tf.Session() as sess:
    
    sess.run(iterator.initializer)
    
    inp, targ = sess.run(next_element)
    
output = rr_Model.predict(inp)

t = range(1000)

plt.plot(t,inp[7,:1000],'k',t,targ[7,:1000]-2,'r',t,output[7,:1000]-2,'b')

plt.show()

## Posprocessing Phase

In [ ]:
Test_dict_ff = dict()
Test_dict_rr = dict()

for _id in range(len(os.listdir())):
    
    file = os.listdir()[_id]
    
    if not file.startswith('I'):
        
        record = np.transpose(np.load(file))
        
        Test_dict_ff[file] = [record[:2,:], record[2,:]]
        Test_dict_rr[file] = [record[:2,:], record[2,:]]


In [ ]:
def crop(array_2D, seqL, ninputs):
    
    results = []
    
    record_seg_dim = int(seqL * ninputs)
    
    offset = 0
    
    for n_time in range(int(180555 / (record_seg_dim))):
        
        record_seg = array_2D[:, offset : offset + record_seg_dim]
        
        results.append(record_seg)
        
        offset += record_seg_dim
    
    return results

In [ ]:
def total_pred(crop_list, Model, Model_type):
    
    if Model_type == 'ff':
        
        dim = 2000
        
    else:
        
        dim = 2100
    
    input_ = np.hstack([crop_list[0][0,:], crop_list[0][1,:]]).reshape(-1,dim)
        
    output = Model.predict(input_) 
    
    for crop_seg in range(1,len(crop_list)):
        
        input_ = np.hstack([crop_list[crop_seg][0,:], crop_list[crop_seg][1,:]]).reshape(-1,dim)
        
        output_ = Model.predict(input_)
        
        output = np.hstack([output, output_])
        
    
    return output

In [ ]:
def get_position(list_values, threshold_value):
    
    pred_array = list_values[0].flatten()
    
    ground_truth = np.array(list_values[1])
    
    
    initial_pos = find_local_peaks(pred_array, 25)
    
    logic_vec = pred_array[initial_pos] >= threshold_value
    
    final_pos_pred = initial_pos[logic_vec]
    
    final_pos_true = np.where(ground_truth == 1)
    
    
    return final_pos_pred, final_pos_true[0]

In [ ]:
def get_comparison(Test_dict, dict_type):
    
    for ECG_file in Test_dict.keys():
    
        reference = rdann(ECG_file[:3] + '_gt', extension = 'atr' )
    
        prediction = rdann(ECG_file[:3] + '_pr_' + dict_type , extension = 'atr')
    
        comparator = compare_annotations(reference.sample, prediction.sample, 12)
    
        string = StringIO()
    
        sys.stdout = string
    
        comparator.print_summary()
    
        Test_dict[ECG_file] = [string.getvalue().split('\n')[6], string.getvalue().split('\n')[7] ]
        
    return Test_dict

In [ ]:
seqL_ff, seqL_rr = 20, 35

fs = 100

time_step_ff, time_step_rr = 5, 0.3

ninputs_rr, ninputs_ff = int(time_step_rr * fs), int(time_step_ff * fs)

for ECG_file in Test_dict_rr.keys():
    
    Test_dict_rr[ECG_file][0] = crop(Test_dict_rr[ECG_file][0], seqL_rr, ninputs_rr)
    
    Test_dict_rr[ECG_file][0] = total_pred(Test_dict_rr[ECG_file][0], rr_Model, 'rr')
    
    Test_dict_rr[ECG_file] = get_position(Test_dict_rr[ECG_file], 0.5)
    
    Test_dict_ff[ECG_file][0] = crop(Test_dict_ff[ECG_file][0], seqL_ff, ninputs_ff)
    
    Test_dict_ff[ECG_file][0] = total_pred(Test_dict_ff[ECG_file][0], ff_Model, 'ff')
    
    Test_dict_ff[ECG_file] = get_position(Test_dict_ff[ECG_file], 0.15)

In [ ]:
for ECG_file in Test_dict_ff.keys():
    
    wrann(ECG_file[:3] + '_pr_ff', 'atr', Test_dict_ff[ECG_file][0], ['N'] * len(Test_dict_ff[ECG_file][0]) )
    
    wrann(ECG_file[:3] + '_pr_rr', 'atr', Test_dict_rr[ECG_file][0], ['N'] * len(Test_dict_rr[ECG_file][0]) )
    
    wrann(ECG_file[:3] + '_gt', 'atr', Test_dict_rr[ECG_file][1], ['N'] * len(Test_dict_rr[ECG_file][1]) ) 
        
    print(ECG_file + ' done')

In [ ]:
Test_dict_rr = get_comparison(Test_dict_rr, 'rr')

Test_dict_ff = get_comparison(Test_dict_ff, 'ff')